<a href="https://colab.research.google.com/github/marielouisehanna/Machine_learning/blob/main/Final%20Project%20MoodifyMe/text_emotions_detection-main/3_BERT_Model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/text_emotions_detection-main.zip -d /content/

Archive:  /content/drive/MyDrive/text_emotions_detection-main.zip
   creating: /content/text_emotions_detection-main/
   creating: /content/text_emotions_detection-main/.idea/
  inflating: /content/text_emotions_detection-main/.idea/.gitignore  
   creating: /content/text_emotions_detection-main/.idea/inspectionProfiles/
  inflating: /content/text_emotions_detection-main/.idea/inspectionProfiles/Project_Default.xml  
  inflating: /content/text_emotions_detection-main/.idea/misc.xml  
  inflating: /content/text_emotions_detection-main/.idea/modules.xml  
  inflating: /content/text_emotions_detection-main/.idea/text_emotions_detection-main.iml  
  inflating: /content/text_emotions_detection-main/1.Preprocessing.ipynb  
  inflating: /content/text_emotions_detection-main/2.BERT_Model1.ipynb  
  inflating: /content/text_emotions_detection-main/3.BERT_Model2.ipynb  
  inflating: /content/text_emotions_detection-main/bert-weights.hdf5  
   creating: /content/text_emotions_detection-main/data/

# Introduction


*   While analyzing the predictions made by the initial BERT model, we noticed that **some 'Neutral' comments were actually not 'Neutral'**
*   It appears that this class has been used like a **"trash" class** when manual labeling was difficult
*   Some samples were labeled with multiple labels, including the 'Neutral' emotion. However, **a sample cannot be 'Neutral' and also include other emotions**
*   Training a model on these **noisy data** could lead to increase the confusion between some emotions
* Let's try and train a new model by **filtering out the 'Neutral'** samples



# 1 - Importing libraries and loading data

## 1.1 - Installing and importing libraries

First, let's install the `transformers` library which contains thousands of pre-trained models, including BERT.

In [3]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install emoji
!pip install contractions

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-x16dfb0v
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-x16dfb0v
  Resolved https://github.com/huggingface/transformers.git to commit 953196a43dae6a3c474165fba7d215fcbc7b7730
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.52.0.dev0-py3-none-any.whl size=11132583 sha256=76117d6ca68dd5440161557333d9d107328950fc2b3126d007ce281e4f300729
  Stored in directory: /tmp/pip-ephem-wheel-cache-itlu94b5/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.3
    Uninstalling transformers-4.50.3:
      Successfully uninstalled transformers-4.50.3
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# Data manipulation libraries
import sys, os
import pandas as pd
import numpy as np
import json

import emoji
import contractions
import re

# Scikit-learn packages
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight

# Packages to define a BERT model
from transformers import TFBertModel, BertTokenizerFast, BertConfig

# Keras and TensorFlow packages
import tensorflow as tf
from tensorflow import keras
from tf_keras import layers
from keras import backend as K
from tf_keras.layers import Input, Dropout, Dense
from tf_keras.models import Model
from tf_keras.optimizers import Adam
from tf_keras.initializers import TruncatedNormal

## 1.2 - Loading datasets and lists of emotions

First, let's load our clean data.

In [8]:
# Importing train, validation and test datasets with preprocessed texts and labels
train_GE = pd.read_csv(r"/content/text_emotions_detection-main/data/train_clean.csv")
val_GE = pd.read_csv(r"/content/text_emotions_detection-main/data/val_clean.csv")
test_GE = pd.read_csv(r"/content/text_emotions_detection-main/data/test_clean.csv")

# Shape validation
print(train_GE.shape)
print(val_GE.shape)
print(test_GE.shape)

(43410, 29)
(5426, 29)
(5427, 29)


Let's also load the lists of emotions from GoEmotions and Ekman taxonomies **excluding the the 'Neutral' emotion** this time.

In [32]:
# Loading emotion labels for GoEmotions taxonomy
with open("/content/text_emotions_detection-main/data/emotions.txt", "r") as file:
    GE_taxonomy = file.read().split("\n")
GE_taxonomy.remove('neutral')
print("Emotions on GoEmotions taxonomy are : \n{}".format(GE_taxonomy))

print()

# Loading emotion labels for Ekman taxonomy
with open("/content/text_emotions_detection-main/data/ekman_labels.txt", "r") as file:
    Ekman_taxonomy = file.read().split("\n")
Ekman_taxonomy.remove('neutral')
print("Emotions on Ekman taxonomy are : \n{}".format(Ekman_taxonomy))

Emotions on GoEmotions taxonomy are : 
['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']

Emotions on Ekman taxonomy are : 
['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']


## 1.3 - Filtering out the 'Neutral' only samples

First, we need to drop the 'Neutral' emotion from all datasets.

In [21]:
train_GE = train_GE.drop(columns=['neutral'], errors='ignore')
val_GE = val_GE.drop(columns=['neutral'], errors='ignore')
test_GE = test_GE.drop(columns=['neutral'], errors='ignore')


Then, we need remove all the samples that have been left without a label.

In [22]:
# Removing samples with only 0 in their labels
train_GE = train_GE.loc[ train_GE.apply(lambda x: sum(x[1:]), axis=1)>0 ]
val_GE = val_GE.loc[ val_GE.apply(lambda x: sum(x[1:]), axis=1)>0 ]
test_GE = test_GE.loc[ test_GE.apply(lambda x: sum(x[1:]), axis=1)>0 ]

# Shape validation
print(train_GE.shape)
print(val_GE.shape)
print(test_GE.shape)

(30587, 28)
(3834, 28)
(3821, 28)


In [23]:
# Preview of data
display(train_GE.head(3))

,Clean_text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise
2,why the fuck is bayless isoing,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,to make her feel threatened,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,dirty southern wankers,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Doing so, we have **decreased the number of samples by nearly 30%** of the original data.

#2 - Modeling : BERT (Bidirectional Encoder Representations from Transformers)

Now we can go ahead and start defining our BERT-based model.

##2.1 - Configuration of the base model

First of all, let's define a `max_length` variable. This variable sets a fixed length of sequences to be fed to our model. Therefore, sequences will be either truncated if larger than this value, or completed using padding if smaller. To avoid truncating, we fix this value according to the largest sample of our data.

In [24]:
# Computing max length of samples
full_text = pd.concat([train_GE['Clean_text'], val_GE['Clean_text'], test_GE['Clean_text']])
max_length = full_text.apply(lambda x: len(x.split())).max()
max_length

48

We are going to use BERT's base model which contains almost 110 M trainable parameters.

Also, in order to match the tokenization and vocabulary used during the training, we are going to use a BERT tokenizer.

In [26]:
# Importing BERT pre-trained model and tokenizer
model_name = 'bert-base-uncased'
config = BertConfig.from_pretrained(model_name, output_hidden_states=False)
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

## 2.2 - Definition of the model architecture

Now that everything is in place, we can create a model based on BERT's main layer, and replace the top layers to reach our main objective (multi-label classification accross **27 possible emotions**).

Our model takes three inputs that result from tokenization:

*   `input_ids`: indices of input sequence tokens in the vocabulary
*   `token_type_ids`: Segment token indices to indicate first and second portions of the inputs.   0 for sentence A and 1 for sentence B
*   `attention mask`: Mask to avoid performing attention on padding token indices.  0 for masked and 1 for not masked



In [27]:
import tensorflow as tf
from transformers import TFBertModel, BertConfig, BertTokenizerFast
from tf_keras.layers import Input, Dropout, Dense
from tf_keras.models import Model
from tf_keras.initializers import TruncatedNormal

def create_model(nb_labels, max_length=48):
    # 1) Keras Inputs (tf.Tensor-backed)
    input_ids      = Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask")
    token_type_ids = Input(shape=(max_length,), dtype=tf.int32, name="token_type_ids")

    # 2) The Hugging Face wrapper (tf.keras.Model)
    bert_model = TFBertModel.from_pretrained("bert-base-uncased")

    # 3) Call it with named args
    outputs = bert_model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids=token_type_ids
    )
    pooled_output = outputs.pooler_output

    # 4) Dropout + classification head
    x = Dropout(bert_model.config.hidden_dropout_prob)(pooled_output)
    logits = Dense(
        nb_labels,
        activation="sigmoid",
        kernel_initializer=TruncatedNormal(stddev=bert_model.config.initializer_range),
        name="emotion"
    )(x)

    # 5) Final model
    return Model(
        inputs=[input_ids, attention_mask, token_type_ids],
        outputs=logits,
        name="BERT_MultiLabel"
    )

We use here a `sigmoid` activation function in the last dense layer that is better suited than a `softmax` activation function. In fact, `softmax` shrinks output probabilities for each label so that the sum of probabilities is 1. In our case, each label (emotion) can independently have a probability between 0 and 1, and `sigmoid` allows that.

We can now create our model using 27 labels and visualize a summary.

In [28]:
# Creating a model instance
model = create_model(27)

# Take a look at the model
model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "BERT_MultiLabel"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 48)]                 0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 48)]                 0         []                            
 )                                                                                                
                                                                                                  
 token_type_ids (InputLayer  [(None, 48)]                 0         []                            
 )                                                                                                
                                                                                    

##2.3 - Data preprocessing and model training

###2.3.1 - Tokenizing data

Let's go ahead and process our data. We will first separate texts from labels in the train, validation and test datasets, and then tokenize the texts using the BERT tokenizer.

In [33]:
# Creating train, validation and test variables
X_train = train_GE['Clean_text']
y_train = train_GE.loc[:, GE_taxonomy].values.astype(float)

X_val = val_GE['Clean_text']
y_val = val_GE.loc[:, GE_taxonomy].values.astype(float)

X_test = test_GE['Clean_text']
y_test = test_GE.loc[:, GE_taxonomy].values.astype(float)

# Tokenizing train data
train_token = tokenizer(
    text = X_train.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length',
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

# Tokenizing valisation data
val_token = tokenizer(
    text = X_val.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length',
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

# Tokenizing test data
test_token = tokenizer(
    text = X_test.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length',
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

In [34]:
# Creating BERT compatible inputs with Input Ids, attention masks and token Ids
train = {'input_ids': train_token['input_ids'], 'attention_mask': train_token['attention_mask'],'token_type_ids': train_token['token_type_ids']}
val = {'input_ids': val_token['input_ids'], 'attention_mask': val_token['attention_mask'],'token_type_ids': val_token['token_type_ids']}
test = {'input_ids': test_token['input_ids'], 'attention_mask': test_token['attention_mask'],'token_type_ids': test_token['token_type_ids']}

During the training phase, we our going to use batches of 16 samples. After each epoch, data will be shuffled. Let's create TensorFlow tensors accordingly.

In [35]:
# Creating TF tensors
train_tensor = tf.data.Dataset.from_tensor_slices((train, y_train)).shuffle(len(train)).batch(16)
val_tensor = tf.data.Dataset.from_tensor_slices((val, y_val)).shuffle(len(val)).batch(16)
test_tensor = tf.data.Dataset.from_tensor_slices((test, y_test)).shuffle(len(test)).batch(16)

### 2.3.2 - Class weights for multi-label and custom loss function

Training requires to monitor the loss function and eventually some other metrics to see how the model behaves throughout the epochs.

Therefore, we need to define a weighted loss function that takes into account  class weights in our multi-label case.

First, we need to compute class weights.

In [36]:
# Function for calculating multilabel class weights
def calculating_class_weights(y_true):
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight(class_weight="balanced", classes=np.unique([0.,1.]), y=y_true[:, i])
    return weights

class_weights = calculating_class_weights(y_train)

Then, we can define a custom crossentropy function in which we multiply the weights.

In [39]:
# Custom loss function for multilabel
from tensorflow.keras import backend as K  # This should fix the issue

def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss

###2.3.3 - Model training

Everything is ready, we can now start training our model.

We chose not to exceed 4 epochs to train our model as it will most likely start to overfit our data.

In [40]:
# Set an optimizer
optimizer = Adam(
    learning_rate=3.e-05,
    )

# Set loss
loss = get_weighted_loss(class_weights)

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss)

# train the model
history = model.fit(train_tensor,
                    epochs=4,
                    validation_data=val_tensor,
                    )

Epoch 1/4
1912/1912 [==============================] - 382s 176ms/step - loss: 0.4442 - val_loss: 0.3418
Epoch 2/4
1912/1912 [==============================] - 339s 177ms/step - loss: 0.2852 - val_loss: 0.3061
Epoch 3/4
1912/1912 [==============================] - 337s 176ms/step - loss: 0.2125 - val_loss: 0.3153
Epoch 4/4
1912/1912 [==============================] - 337s 176ms/step - loss: 0.1657 - val_loss: 0.3358


In [41]:
model.save("my_model2.h5")

/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [42]:
model.save('my_model2.keras')

/usr/local/lib/python3.11/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


## 2.4 - Model evaluation

### 2.4.1 - Evaluation on GoEmotions taxonomy

In [43]:
# Save model weights
model.save_weights('bert-weights.hdf5')

Let's generate predictions on test data.

In [44]:
# Making probability predictions on test data
y_pred_proba = model.predict(test)

120/120 [==============================] - 16s 102ms/step


When making predictions, we only generate probabilities associated with each label. To predict actual labels, we need to add an additional step that transforms these probabilities into labels given a certain threshold.

We define a function to do so with a default threshold set to 0.8.

In [45]:
# from probabilities to labels using a given threshold
def proba_to_labels(y_pred_proba, threshold=0.8):

    y_pred_labels = np.zeros_like(y_pred_proba)

    for i in range(y_pred_proba.shape[0]):
        for j in range(y_pred_proba.shape[1]):
            if y_pred_proba[i][j] > threshold:
                y_pred_labels[i][j] = 1
            else:
                y_pred_labels[i][j] = 0

    return y_pred_labels

In [46]:
# Generate labels
y_pred_labels = proba_to_labels(y_pred_proba)

Let's evaluate these predictions using the evaluation function we defined in the previous notebooks.

In [47]:
# Model evaluation function
def model_eval(y_true, y_pred_labels, emotions):

    # Defining variables
    precision = []
    recall = []
    f1 = []

    # Per emotion evaluation
    idx2emotion = {i: e for i, e in enumerate(emotions)}

    for i in range(len(emotions)):

        # Computing precision, recall and f1-score
        p, r, f1_score, _ = precision_recall_fscore_support(y_true[:, i], y_pred_labels[:, i], average="binary")

        # Append results in lists
        precision.append(round(p, 2))
        recall.append(round(r, 2))
        f1.append(round(f1_score, 2))

    # Macro evaluation
    macro_p, macro_r, macro_f1_score, _ = precision_recall_fscore_support(y_true, y_pred_labels, average="macro")

    # Append results in lists
    precision.append(round(macro_p, 2))
    recall.append(round(macro_r, 2))
    f1.append(round(macro_f1_score, 2))

    # Converting results to a dataframe
    df_results = pd.DataFrame({"Precision":precision, "Recall":recall, 'F1':f1})
    df_results.index = emotions+['MACRO-AVERAGE']

    return df_results

In [48]:
# Model evaluation
model_eval(y_test, y_pred_labels, GE_taxonomy)

,Precision,Recall,F1
admiration,0.71,0.71,0.71
amusement,0.80,0.91,0.85
anger,0.44,0.65,0.52
annoyance,0.39,0.54,0.46
approval,0.54,0.49,0.51
caring,0.39,0.57,0.47
confusion,0.35,0.73,0.47
curiosity,0.55,0.85,0.67
desire,0.36,0.69,0.47
disappointment,0.28,0.44,0.35


Looking at the results, we see that this model performs better than the previous one. It looks like **removing the noise brought by the 'Neutral' emotion helped to better distinguish the other emotions.**

### 2.4.2 - Threshold optimization

In the initial evaluation, we set an aribitrary threshold. However, we can also choose a threshold that maximizes a certain metric.

We define a function that tests a certain number of possible thresholds, and returns the best threshold together with the best predicted labels and best macro f1-score.

In [50]:
# Function that computes labels from probabilities and optimizes the threshold that maximizes f1-score
def proba_to_labels_opt(y_true, y_pred_proba):

    '''
    Inputs:
        y_true: Ground truth labels
        y_pred_proba: predicted probabilities

    Outputs :
        best_y_pred_labels: preticted labels associated with best threshold
        best_t: best threshold
        best_macro_f1: macro f1-score associated with predicted labels
    '''

    # range of possible thresholds
    thresholds = np.arange(0.7, 0.99, 0.01)

    # Computing threshold that maximizes macro f1-score
    best_y_pred_labels = np.zeros_like(y_pred_proba)
    best_t = 0
    best_macro_f1 = 0

    # Iterating through possible thresholds
    for t in thresholds:

        y_pred_labels = proba_to_labels(y_pred_proba, t)

        _, _, macro_f1, _ = precision_recall_fscore_support(y_true, y_pred_labels, average="macro")

        if macro_f1 > best_macro_f1:
            best_macro_f1 = macro_f1
            best_t = t
            best_y_pred_labels = y_pred_labels

    return best_y_pred_labels, best_t, best_macro_f1

We can now apply this function to our predicted probabilities and compute optimized label predictions.

In [51]:
# Compute label predictions and corresponding optimal thresholds
y_pred_labels_opt, threshold_opt, macro_f1_opt = proba_to_labels_opt(y_test, y_pred_proba)
print("The model's threshold is {}".format(threshold_opt))
print("The model's best macro-f1 is {}".format(macro_f1_opt))

The model's threshold is 0.9200000000000002
The model's best macro-f1 is 0.5262833625465558


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [52]:
# Model evaluation : Precision, Recall, F-score
model_eval(y_test, y_pred_labels_opt, GE_taxonomy)

,Precision,Recall,F1
admiration,0.77,0.57,0.66
amusement,0.85,0.83,0.84
anger,0.55,0.51,0.52
annoyance,0.62,0.22,0.32
approval,0.64,0.30,0.40
caring,0.53,0.48,0.50
confusion,0.49,0.52,0.51
curiosity,0.65,0.77,0.71
desire,0.53,0.59,0.56
disappointment,0.38,0.26,0.31


**Optimizing the threshold** helped us to **slightly improve** the model predictions.

### 2.4.4 - Indirect evaluation on Ekman taxonomy by mapping predictions

Until now, we have only evaluated our model on the GoEmotions taxonomy.

As a reference, we can try to map the true and predicted emotions to the Ekman taxonomy and see how our model performs.

We have already defined the Ekman taxonomy earlier.

Let's define a function that transforms labels from GoEmotions to Ekman taxonomy.

In [53]:
# Function thats maps predictions on GoEmotions taxonomy to Ekman taxonomy
def GE_to_Ekman(GE_labels):

    # Create a dataframe of GoEmotions labels
    df_GE = pd.DataFrame(GE_labels, columns=GE_taxonomy)

    # Create an empty dataframe of Ekman labels
    df_Ekman  = pd.DataFrame(np.zeros((len(GE_labels), len(Ekman_taxonomy))), columns=Ekman_taxonomy)

    for i in range(len(df_GE)):

        if df_GE.loc[i,['anger', 'annoyance', 'disapproval']].sum() >= 1:
            df_Ekman.loc[i,'anger'] = 1

        if df_GE.loc[i,'disgust'].sum() >= 1:
            df_Ekman.loc[i,'disgust'] = 1

        if df_GE.loc[i,['fear', 'nervousness']].sum() >= 1:
            df_Ekman.loc[i,'fear'] = 1

        if df_GE.loc[i,['joy', 'amusement', 'approval', 'excitement', 'gratitude',
                        'love', 'optimism', 'relief', 'pride', 'admiration', 'desire','caring']].sum() >= 1:
            df_Ekman.loc[i,'joy'] = 1

        if df_GE.loc[i,['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse']].sum() >= 1:
            df_Ekman.loc[i,'sadness'] = 1

        if df_GE.loc[i,['surprise', 'realization', 'confusion', 'curiosity']].sum() >= 1:
            df_Ekman.loc[i,'surprise'] = 1

    return df_Ekman.values

We can now apply our function and evaluate the predictions

In [54]:
# Mapping GoEmotion labels to Ekman labels (true and predictions)
y_test_Ekman = GE_to_Ekman(y_test)
y_pred_labels_Ekman = GE_to_Ekman(y_pred_labels_opt)

# Evaluation
model_eval(y_test_Ekman, y_pred_labels_Ekman, Ekman_taxonomy)

,Precision,Recall,F1
anger,0.79,0.48,0.60
disgust,0.44,0.65,0.52
fear,0.61,0.70,0.65
joy,0.92,0.83,0.87
sadness,0.64,0.59,0.61
surprise,0.70,0.71,0.71
MACRO-AVERAGE,0.68,0.66,0.66


## 2.5 - Make predictions

To make predictions on a new sample, it needs to be processed using all the different precessing steps we used.

In [55]:
# Retrieving initial preprocessings
def preprocess_corpus(x):

    # Adding a space between words and punctation
    x = re.sub( r'([a-zA-Z\[\]])([,;.!?])', r'\1 \2', x)
    x = re.sub( r'([,;.!?])([a-zA-Z\[\]])', r'\1 \2', x)

    # Demojize
    x = emoji.demojize(x)

    # Expand contraction
    x = contractions.fix(x)

    # Lower
    x = x.lower()

    #correct some acronyms/typos/abbreviations
    x = re.sub(r"lmao", "laughing my ass off", x)
    x = re.sub(r"amirite", "am i right", x)
    x = re.sub(r"\b(tho)\b", "though", x)
    x = re.sub(r"\b(ikr)\b", "i know right", x)
    x = re.sub(r"\b(ya|u)\b", "you", x)
    x = re.sub(r"\b(eu)\b", "europe", x)
    x = re.sub(r"\b(da)\b", "the", x)
    x = re.sub(r"\b(dat)\b", "that", x)
    x = re.sub(r"\b(dats)\b", "that is", x)
    x = re.sub(r"\b(cuz)\b", "because", x)
    x = re.sub(r"\b(fkn)\b", "fucking", x)
    x = re.sub(r"\b(tbh)\b", "to be honest", x)
    x = re.sub(r"\b(tbf)\b", "to be fair", x)
    x = re.sub(r"faux pas", "mistake", x)
    x = re.sub(r"\b(btw)\b", "by the way", x)
    x = re.sub(r"\b(bs)\b", "bullshit", x)
    x = re.sub(r"\b(kinda)\b", "kind of", x)
    x = re.sub(r"\b(bruh)\b", "bro", x)
    x = re.sub(r"\b(w/e)\b", "whatever", x)
    x = re.sub(r"\b(w/)\b", "with", x)
    x = re.sub(r"\b(w/o)\b", "without", x)
    x = re.sub(r"\b(doj)\b", "department of justice", x)

    # replace some words with multiple occurences of a letter, example "coooool" turns into --> cool
    x = re.sub(r"\b(j+e{2,}z+e*)\b", "jeez", x)
    x = re.sub(r"\b(co+l+)\b", "cool", x)
    x = re.sub(r"\b(g+o+a+l+)\b", "goal", x)
    x = re.sub(r"\b(s+h+i+t+)\b", "shit", x)
    x = re.sub(r"\b(o+m+g+)\b", "omg", x)
    x = re.sub(r"\b(w+t+f+)\b", "wtf", x)
    x = re.sub(r"\b(w+h+a+t+)\b", "what", x)
    x = re.sub(r"\b(y+e+y+|y+a+y+|y+e+a+h+)\b", "yeah", x)
    x = re.sub(r"\b(w+o+w+)\b", "wow", x)
    x = re.sub(r"\b(w+h+y+)\b", "why", x)
    x = re.sub(r"\b(s+o+)\b", "so", x)
    x = re.sub(r"\b(f)\b", "fuck", x)
    x = re.sub(r"\b(w+h+o+p+s+)\b", "whoops", x)
    x = re.sub(r"\b(ofc)\b", "of course", x)
    x = re.sub(r"\b(the us)\b", "usa", x)
    x = re.sub(r"\b(gf)\b", "girlfriend", x)
    x = re.sub(r"\b(hr)\b", "human ressources", x)
    x = re.sub(r"\b(mh)\b", "mental health", x)
    x = re.sub(r"\b(idk)\b", "i do not know", x)
    x = re.sub(r"\b(gotcha)\b", "i got you", x)
    x = re.sub(r"\b(y+e+p+)\b", "yes", x)
    x = re.sub(r"\b(a*ha+h[ha]*|a*ha +h[ha]*)\b", "haha", x)
    x = re.sub(r"\b(o?l+o+l+[ol]*)\b", "lol", x)
    x = re.sub(r"\b(o*ho+h[ho]*|o*ho +h[ho]*)\b", "ohoh", x)
    x = re.sub(r"\b(o+h+)\b", "oh", x)
    x = re.sub(r"\b(a+h+)\b", "ah", x)
    x = re.sub(r"\b(u+h+)\b", "uh", x)

    # Handling emojis
    x = re.sub(r"<3", " love ", x)
    x = re.sub(r"xd", " smiling_face_with_open_mouth_and_tightly_closed_eyes ", x)
    x = re.sub(r":\)", " smiling_face ", x)
    x = re.sub(r"^_^", " smiling_face ", x)
    x = re.sub(r"\*_\*", " star_struck ", x)
    x = re.sub(r":\(", " frowning_face ", x)
    x = re.sub(r":\^\(", " frowning_face ", x)
    x = re.sub(r";\(", " frowning_face ", x)
    x = re.sub(r":\/",  " confused_face", x)
    x = re.sub(r";\)",  " wink", x)
    x = re.sub(r">__<",  " unamused ", x)
    x = re.sub(r"\b([xo]+x*)\b", " xoxo ", x)
    x = re.sub(r"\b(n+a+h+)\b", "no", x)

    # Handling special cases of text
    x = re.sub(r"h a m b e r d e r s", "hamberders", x)
    x = re.sub(r"b e n", "ben", x)
    x = re.sub(r"s a t i r e", "satire", x)
    x = re.sub(r"y i k e s", "yikes", x)
    x = re.sub(r"s p o i l e r", "spoiler", x)
    x = re.sub(r"thankyou", "thank you", x)
    x = re.sub(r"a^r^o^o^o^o^o^o^o^n^d", "around", x)

    # Remove special characters and numbers replace by space + remove double space
    x = re.sub(r"\b([.]{3,})"," dots ", x)
    x = re.sub(r"[^A-Za-z!?_]+"," ", x)
    x = re.sub(r"\b([s])\b *","", x)
    x = re.sub(r" +"," ", x)
    x = x.strip()

    return x

Now we can define a prediction function that takes one or more samples, and outputs the detected emotions from the model.

In [56]:
def predict_samples(text_samples, model, threshold):

    # Text preprocessing and cleaning
    text_samples_clean = [preprocess_corpus(text) for text in text_samples]

    # Tokenizing train data
    samples_token = tokenizer(
        text = text_samples_clean,
        add_special_tokens = True,
        max_length = max_length,
        truncation = True,
        padding = 'max_length',
        return_tensors = 'tf',
        return_token_type_ids = True,
        return_attention_mask = True,
        verbose = True,
    )

    # Preparing to feed the model
    samples = {'input_ids': samples_token['input_ids'],
               'attention_mask': samples_token['attention_mask'],
               'token_type_ids': samples_token['token_type_ids']
              }

    # Probability predictions
    samples_pred_proba = model.predict(samples)

    # Label prediction using threshold
    samples_pred_labels = proba_to_labels(samples_pred_proba)

    samples_pred_labels_df = pd.DataFrame(samples_pred_labels)
    samples_pred_labels_df = samples_pred_labels_df.apply(lambda x: [GE_taxonomy[i] for i in range(len(x)) if x[i]==1], axis=1)

    #return list(samples_pred_labels_df)
    return pd.DataFrame({"Text":text_samples, "Emotions":list(samples_pred_labels_df)})

Let's try on few examples.

In [57]:
# Predict samples
predict_samples(["My favourite food is anything I didn't have to cook myself", "are you kiddin me ??!!", "red"], model, threshold_opt)

1/1 [==============================] - 0s 55ms/step


,Text,Emotions
0,My favourite food is anything I didn't have to...,[joy]
1,are you kiddin me ??!!,"[confusion, curiosity]"
2,red,[]


Although the score is not very high, we see that the model **detects emotions that coherent**. Also, **when entering a neutral text such as "red", the model does not detect any emotion**.

# Conclusion

*   In this notebook, we verified our assumption : **filtering out the 'Neutral' samples from the data allows to improve the model**.

*   Not only **we better distinguish actual emotions**, but **we can also detect 'Neutral' comments**, **without teaching our model what is a 'Neutral' comment**.

In [58]:
!cp -r /content/my_model2.h5 /content/drive/My\ Drive/
!cp -r /content/my_model2.keras /content/drive/My\ Drive/


